In [ ]:
# selector : div.epub.ui-collapsible h2.ui-collapsible-heading a

In [6]:
import json, os, pprint, time, re
from pprint import pprint as pp
from urllib import parse
import requests as req
from bs4 import BeautifulSoup as bs

In [13]:
# 隨機取得user agent
from fake_useragent import UserAgent
ua = UserAgent(cache=True)

print(ua.random)

Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.62 Safari/537.36


In [61]:
'''放置 金庸小說 metadata 的資訊'''
listData = []

'''小庸小說的網址'''
url = 'https://www.bookwormzz.com/zh/'

'''設定標頭'''
my_headers = {
    'user-agent': ua.random
}

# 沒有放置 txt 檔的資料夾，就建立起來
folderPath = 'jinyong'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 取得小說#book_toc目錄頁的超連結
def getMainLinks():
    response = req.get(url, headers = my_headers)
    soup = bs(response.text, 'lxml')
# 所有a的超連結，是一個集合list
    a_elements = soup.select('a[data-ajax="false"]')
    for a in a_elements:
        mainUrl = {
            "title":a.get_text(),
            "link":url + parse.unquote(a.get("href")) + "#book_toc"
        }
        listData.append(mainUrl)
    pp(listData)

# 取得目錄頁中所有超連結
def getSubLink():
    for i in range(len(listData)):
        #沒有sub屬性，則建立為了放置各本小說中的章回資訊
        if "sub" not in listData[i]:
            listData[i]["sub"] = []
        response = req.get(listData[i]['link'],headers = my_headers)
        soup = bs(response.text, 'lxml')
        a_elements = soup.select('div[data-theme="b"][data-content-theme="c"] a[rel="external"]')
        # 若是走訪網頁時，選擇不到特定的元素，視為沒有資料，使用continue
        if len(a_elements) > 0:
            
            for a in a_elements:
                subUrl = {
                    "title":a.get_text(),
                    "sub_link":url + parse.unquote(a.get("href"))
                }
                listData[i]["sub"].append(subUrl)
        else:
            continue
    pp(listData)
# 建立金庸小說的json檔
def saveJson():
    fp = open(f"{folderPath}/jinyong.json", "w", encoding="utf-8")
    fp.write(json.dumps(listData, ensure_ascii=False, indent=4)) # 儲存單純中文
    fp.close()
    
# 將金庸小說所有章回的內容，各自寫到txt與json中
def writeTxt():
    # 稍後建立train.json前的程式變數
    listContent = []
    
    # 開啟金庸小說json檔
    fp = open(f"{folderPath}/jinyong.json", "r", encoding="utf-8")
    strJson = fp.read()
    fp.close()
    
    #如果沒有放置txt檔的資料夾就建立起來
    folderPath_text = f"{folderPath}/jinyong_txt_reading_version"
    if not os.path.exists(folderPath_text):
        os.makedirs(folderPath_text)
        
    # 走訪所有章回小說的文字內容
    listResult = json.loads(strJson)
    for i in range(len(listResult)):
        for j in range(len(listResult[i]['sub'])):
            response = req.get(listResult[i]['sub'][j]['sub_link'], headers=my_headers)
            soup = bs(response.text, 'lxml')
            div = soup.select_one('div#html > div')
            strContent = div.get_text()
            
            # 取代不必要的符號及文字
            #strContent = re.sub(r"\s|\r|\n|　| ","", strContent)
            strContent = re.sub(r"\s|\r|　| ","\n", strContent)
            
            # 決定txt檔案名稱 e.g.,金庸《射鵰英雄傳一》_第一回 風雪驚變.txt
            fileName = f"{listResult[i]['title']}_{listResult[i]['sub'][j]['title']}.txt"
            
            # 將小說內容存到txt中
            fp = open(f"{folderPath_text}/{fileName}", "w", encoding="utf-8")
            fp.write(strContent)
            fp.close()
            
            # 額外將小說內容放到list當中，建立train.json
            listContent.append(strContent)
    
    # 延伸之後的議題，在此建立訓練資料
    fp = open(f"{folderPath}/train.json", "w", encoding="utf-8")
    fp.write = (json.dumps(listContent, ensure_ascii=False, indent=4))
    fp.close()
    

In [62]:
if __name__ == "__main__":
    #getMainLinks()
    #getSubLink()
    #saveJson()
    time1 = time.time()
    writeTxt()
    print(f"執行總花費時間:{time.time()-time1}秒")

執行總花費時間:347.1872470378876秒


In [58]:
folderPath = 'test_novel'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
fp = open(f"jinyong/jinyong.json", "r", encoding="utf-8")
strJson = fp.read()
fp.close()
listResult = json.loads(strJson)
single_response = req.get(listResult[0]['sub'][0]['sub_link'], headers=my_headers)

soup = bs(single_response.text, 'lxml')
div = soup.select_one('div#html > div')
strContent = div.get_text()
strContent = re.sub(r"\s|\r|　| ","\n", strContent)

fileName = f"{listResult[0]['title']}_{listResult[0]['sub'][0]['title']}.txt"

fp = open(f"test_novel/{fileName}", "w", encoding="utf-8")
fp.write(strContent)
fp.close()